In [63]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
os.getcwd()
df = pd.read_excel("Telco_customer_churn.xlsx")
df.head

<bound method NDFrame.head of       CustomerID  Count        Country       State          City  Zip Code  \
0     3668-QPYBK      1  United States  California   Los Angeles     90003   
1     9237-HQITU      1  United States  California   Los Angeles     90005   
2     9305-CDSKC      1  United States  California   Los Angeles     90006   
3     7892-POOKP      1  United States  California   Los Angeles     90010   
4     0280-XJGEX      1  United States  California   Los Angeles     90015   
...          ...    ...            ...         ...           ...       ...   
7038  2569-WGERO      1  United States  California       Landers     92285   
7039  6840-RESVB      1  United States  California      Adelanto     92301   
7040  2234-XADUH      1  United States  California         Amboy     92304   
7041  4801-JZAZL      1  United States  California  Angelus Oaks     92305   
7042  3186-AJIEK      1  United States  California  Apple Valley     92308   

                    Lat Long   La

In [54]:
df.isnull().sum()
#Here we can observe that there are no missing values in the dataset except for the Churn reason column for customers who did not churn. 
#Null values of 5174 churns mean that they continued their subscription at Telco.

df.columns


df.head

<bound method NDFrame.head of       CustomerID  Count        Country       State          City  Zip Code  \
0     3668-QPYBK      1  United States  California   Los Angeles     90003   
1     9237-HQITU      1  United States  California   Los Angeles     90005   
2     9305-CDSKC      1  United States  California   Los Angeles     90006   
3     7892-POOKP      1  United States  California   Los Angeles     90010   
4     0280-XJGEX      1  United States  California   Los Angeles     90015   
...          ...    ...            ...         ...           ...       ...   
7038  2569-WGERO      1  United States  California       Landers     92285   
7039  6840-RESVB      1  United States  California      Adelanto     92301   
7040  2234-XADUH      1  United States  California         Amboy     92304   
7041  4801-JZAZL      1  United States  California  Angelus Oaks     92305   
7042  3186-AJIEK      1  United States  California  Apple Valley     92308   

                    Lat Long   La

In [55]:
df.dtypes

CustomerID            object
Count                  int64
Country               object
State                 object
City                  object
Zip Code               int64
Lat Long              object
Latitude             float64
Longitude            float64
Gender                object
Senior Citizen        object
Partner               object
Dependents            object
Tenure Months          int64
Phone Service         object
Multiple Lines        object
Internet Service      object
Online Security       object
Online Backup         object
Device Protection     object
Tech Support          object
Streaming TV          object
Streaming Movies      object
Contract              object
Paperless Billing     object
Payment Method        object
Monthly Charges      float64
Total Charges         object
Churn Label           object
Churn Value            int64
Churn Score            int64
CLTV                   int64
Churn Reason          object
dtype: object

In [56]:
#Here, we are replacing " " with not a number. 
df["Total Charges"] = df["Total Charges"].replace(" ", np.nan)
#The following line turns the column "Total Charges" to float. 
df["Total Charges"] = df["Total Charges"].astype(float)

yes_no_cols = [
    'Partner', 'Dependents', 'Phone Service', 'Online Security',
    'Online Backup', 'Device Protection', 'Tech Support',
    'Streaming TV', 'Streaming Movies', 'Paperless Billing'
]

for col in yes_no_cols:
    df[col] = df[col].map({'Yes': 1, 'No': 0})


/var/folders/q_/q_8374nd2lz33xgn_4vmrscc0000gn/T/ipykernel_84041/1998270704.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["Total Charges"] = df["Total Charges"].replace(" ", np.nan)


In [57]:
df["Total Charges"].dtype


dtype('float64')

In [58]:
df['Senior Citizen'] = df['Senior Citizen'].map({'Yes': 1, 'No': 0})


In [59]:
df = df.drop(columns=[
    'CustomerID',   # ID only
    'Count',        # always 1
    'Lat Long',     # duplicate of Lat/Long
    'CLTV',         # leaky
    'Churn Reason'  # leaky
])


In [60]:
df = df.drop(columns=['Zip Code', 'Latitude', 'Longitude'])


In [61]:
features_to_use = [
    'Gender',
    'Senior Citizen',
    'Partner',
    'Dependents',
    'Tenure Months',
    'Phone Service',
    'Multiple Lines',
    'Internet Service',
    'Online Security',
    'Online Backup',
    'Device Protection',
    'Tech Support',
    'Streaming TV',
    'Streaming Movies',
    'Contract',
    'Paperless Billing',
    'Payment Method',
    'Monthly Charges',
    'Total Charges'
]
y = df['Churn Value']

X = df[features_to_use]
#This step readies for the data frame for one-hot encoding 
X = pd.get_dummies(X, drop_first=True)


In [64]:
X.head()


,Senior Citizen,Partner,Dependents,Tenure Months,Phone Service,Online Security,Online Backup,Device Protection,Tech Support,Streaming TV,...,Gender_Male,Multiple Lines_No phone service,Multiple Lines_Yes,Internet Service_Fiber optic,Internet Service_No,Contract_One year,Contract_Two year,Payment Method_Credit card (automatic),Payment Method_Electronic check,Payment Method_Mailed check
0,0,0,0,2,1,1.0,1.0,0.0,0.0,0.0,...,True,False,False,False,False,False,False,False,False,True
1,0,0,1,2,1,0.0,0.0,0.0,0.0,0.0,...,False,False,False,True,False,False,False,False,True,False
2,0,0,1,8,1,0.0,0.0,1.0,0.0,1.0,...,False,False,True,True,False,False,False,False,True,False
3,0,1,1,28,1,0.0,0.0,1.0,1.0,1.0,...,False,False,True,True,False,False,False,False,True,False
4,0,0,1,49,1,0.0,1.0,1.0,0.0,1.0,...,True,False,True,True,False,False,False,False,False,False


In [66]:
#Splitting the data frame into test & train
y = df['Churn Value']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
